In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
from urllib.request import urlopen as uReq
from datetime import datetime

In [2]:
p_url = "https://bidplus.gem.gov.in/bidresultlists"

In [3]:
bid_no = []
bid_no_true = []
bid_no_set = set(bid_no_true)
bid_doc = []
bid_status = []
bid_item = []
qty_reqd = []
dept_addrs = []
start_date = []
end_date = []
bid_res = []

in_page_bid_no_now = []
in_page_bid_no = []
in_page_bid_status = []
in_page_qty = []
in_page_bid_life_cycle = []
in_page_bid_validity = []
in_page_bid_start_Dt = []
in_page_bid_end_Dt = []
in_page_name = []
in_page_address = []
in_page_ministry = []
in_page_dept = []
in_page_orgnzn = []
in_page_office = []

In [4]:
sno=[] 
seller_name = []
make = []
model = []
participated_on = []
emd_status = []
tech_status = []
in_page_bid_no_tech_eva = []
head_set_1 = set(['EMD Status','Participated On','S. No.','Seller Name','Status'])
head_set_2 = set(['Offered Item','Participated On','S. No.','Seller Name','Status'])
head_set_3= set(['EMD Status','Offered Item','Participated On','S. No.','Seller Name','Status'])
head_set_4 = set(['Participated On','S. No.','Seller Name','Status'])

In [5]:
sn =[] 
seller_n = []
offered_item = []
total_price = []
awarded = []
in_page_bid_no_fin_eva = []

In [6]:
tech_eva = pd.DataFrame(list(zip(in_page_bid_no_tech_eva,sno,seller_name,make,model,participated_on,tech_status)),columns = ['In Page Bid No','SNO','Seller Name','Make','Model','Participated On','Tecnical Status'])
fin_eva = pd.DataFrame(list(zip(in_page_bid_no_fin_eva,sn,seller_n,offered_item,total_price,awarded)),columns = ['In Page Bid No','SNO','Seller Name','Offered Item','Total Price','Awarded'])
bid_table = pd.DataFrame(list(zip(bid_no_true,bid_doc,bid_status,bid_item,qty_reqd,dept_addrs,start_date,end_date,bid_res,in_page_bid_no,in_page_bid_status,in_page_qty,in_page_bid_life_cycle,in_page_bid_validity,in_page_bid_start_Dt,in_page_bid_end_Dt,in_page_name,in_page_address,in_page_ministry,in_page_dept,in_page_orgnzn,in_page_office)), columns = ['Bid No','Bid Doc URL','Bid Status', 'Bid Item', 'Qty Reqd', 'Dept Address', 'Start Date', 'End Date', 'Bid Result', 'Inpage Bid No', 'Inpage_bid_status', 'Inpage Qty', 'Inpage Bid Life Cycle', 'Inpage Bid Validity' , 'Inpage Bid Start_Dt', 'Inpage Bid End_Dt' , 'Inpage Name', 'Inpage Address' , 'Inpage Ministry', 'Inpage dept', 'Inpage organization', 'Inpage Office'] )    

In [7]:
def last_few_pages_parser(no_of_pages = 20):
    uClient = uReq(p_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,"html.parser")
    last_url = 'https://bidplus.gem.gov.in'+ page_soup.findAll("ul",{"class":"pagination"})[1].findAll('li')[5].a['href']
    last_page_no = int(last_url[-4:])
    i = last_page_no - 2500
    c = 0
    all_page_soup = []
    all_containers = []
    while(i> last_page_no - 2500 - no_of_pages):
        try:
            n_url = "https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no="+str(i)
            containers = []
            uClient = uReq(n_url)
            page_html = uClient.read()
            uClient.close()
            time.sleep(3)
            page_soup = soup(page_html,"html.parser")
            all_page_soup.append(page_soup)
            containers = page_soup.findAll("div",{"class":"border block"})
            
            c=c+1
            print(c,' ',n_url,' ',len(containers))
            all_containers.append(containers)        
            i=i-1
        except ConnectionResetError:
            print('Connection Error - ByPassing')
            pass
        except URLError:
            print('URL Error - Bypassing')
    return all_containers
        

In [9]:
all_containers = last_few_pages_parser(5)
len(all_containers)

1   https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=-180   10
2   https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=-181   10
3   https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=-182   10
4   https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=-183   10
5   https://bidplus.gem.gov.in/bidresultlists?bidresultlists&page_no=-184   10


5

In [10]:
x = datetime.now()
timenow = datetime(x.year,x.month,x.day,x.hour,x.minute,x.second)
time_until = datetime(2019, 11, 30, 16, 0, 5)
try:
    #while(timenow<time_until):
        #all_containers = last_few_pages_parser(500)
        counter = 0
        for containers in all_containers:
            for container in containers:
                try:
                    counter = counter + 1
                    print(counter)
                    bid_or_ra = container.div.p.text
                    if(bid_or_ra[1:7] == 'BID NO'):
                        status = container.div.find("p",{"class":"pull-right"}).span.text
                        #print(status)
                        if(status == 'Bid Award'):
                            bid_no_now = container.div.p.a.text 
                            bid_no.append(bid_no_now)                
                            if bid_no_now not in bid_no_set:
                                    print(bid_no_now)
                                    bid_no_true.append(bid_no_now)
                                    bid_no_set = set(bid_no_true)
                                    bid_status.append(status)
                                    bid_doc.append('https://bidplus.gem.gov.in' + container.div.p.a['href'])                   
                                    bid_item.append(container.div.find_next('div').find_next('div').p.span.text)
                                    qty_reqd.append(container.div.find_next('div').find_next('div').p.find_next('p').span.text)
                                    dept_addrs.append(container.div.find_next('div').find_next('div').find_next('div').find_next('div').p.find_next('p').text.strip())
                                    start_date.append(container.div.find_next('div').find_next('div').find_next('div').find_next('div').find_next('div').find_next('div').p.span.text)
                                    end_date.append(container.div.find_next('div').find_next('div').find_next('div').find_next('div').find_next('div').find_next('div').p.find_next('p').span.text)

                                    in_page_url = 'https://bidplus.gem.gov.in'+container.findAll("div",{"class":"col-md-4 pull-right"})[0].a['href']                        
                                    bid_res.append(in_page_url)                    
                                    in_page_uClient = uReq(in_page_url)
                                    in_page_html = in_page_uClient.read()
                                    in_page_uClient.close()
                                    time.sleep(3)
                                    inpage_soup = soup(in_page_html,"html.parser")

                                    in_page_bid_no_now = inpage_soup.findAll("div",{"class":"block_bid_no"})[0].p.span.text
                                    if bid_no_true[-1] == in_page_bid_no_now:
                                        in_page_bid_no.append(in_page_bid_no_now)
                                        in_page_bid_status.append(inpage_soup.findAll("div",{"class":"col-block"})[0].p.span.text)
                                        in_page_qty.append(inpage_soup.findAll("div",{"class":"col-block"})[0].p.find_next('p').span.text)
                                        in_page_bid_life_cycle.append(inpage_soup.findAll("div",{"class":"col-block"})[0].p.find_next('p').find_next('p').span.text)
                                        in_page_bid_validity.append(inpage_soup.findAll("div",{"class":"col-block"})[0].p.find_next('p').find_next('p').find_next('p').span.text)
                                        in_page_bid_start_Dt.append(inpage_soup.findAll("div",{"class":"col-block"})[1].p.span.text)
                                        in_page_bid_end_Dt.append(inpage_soup.findAll("div",{"class":"col-block"})[1].p.find_next('p').span.text)
                                        in_page_name.append(inpage_soup.findAll("div",{"class":"col-block"})[3].p.span.text)
                                        in_page_address.append(inpage_soup.findAll("div",{"class":"col-block"})[3].p.find_next('p').span.text.strip())
                                        in_page_ministry.append(inpage_soup.findAll("div",{"class":"col-block"})[4].p.span.text)
                                        in_page_dept.append(inpage_soup.findAll("div",{"class":"col-block"})[4].p.find_next('p').span.text)
                                        in_page_orgnzn.append(inpage_soup.findAll("div",{"class":"col-block"})[5].p.span.text)
                                        in_page_office.append(inpage_soup.findAll("div",{"class":"col-block"})[5].p.find_next('p').span.text)

                                        table = inpage_soup.findAll("table",{"class":"table"})[0].tbody.tr
                                        heads = inpage_soup.findAll("table",{"class":"table"})[0].thead.findAll('th')
                                        heading_set = []
                                        for i in heads:
                                            heading_set.append(i.text)
                                        heading_set= set(heading_set)
                                        while(len(table)!=0): 
                                            sno.append(table.td.text)
                                            seller_name.append(table.td.find_next('td').text.replace('\xa0\xa0\xa0Under PMA','').strip())
                                            in_page_bid_no_tech_eva.append(inpage_soup.findAll("div",{"class":"block_bid_no"})[0].p.span.text)
                                            if heading_set == head_set_2:
                                                make.append(table.td.find_next('td').find_next('td').p.text.replace('Make :','').strip())  
                                                model.append(table.td.find_next('td').find_next('td').p.find_next('p').text.replace('Model :','').strip())    
                                                participated_on.append(table.td.find_next('td').find_next('td').find_next('td').text)
                                                emd_status.append(None)
                                                tech_status.append(table.td.find_next('td').find_next('td').find_next('td').find_next('td').text.replace('\n',''))

                                            elif heading_set == head_set_4:
                                                make.append(None)
                                                model.append(None)
                                                participated_on.append(table.td.find_next('td').find_next('td').text)
                                                emd_status.append(None)
                                                tech_status.append(table.td.find_next('td').find_next('td').find_next('td').text.replace('\n',''))

                                            elif heading_set == head_set_3:
                                                make.append(table.td.find_next('td').find_next('td').p.text.replace('Make :','').strip())  
                                                model.append(table.td.find_next('td').find_next('td').p.find_next('p').text.replace('Model :','').strip())    
                                                participated_on.append(table.td.find_next('td').find_next('td').find_next('td').text)
                                                emd_status.append(table.td.find_next('td').find_next('td').find_next('td').find_next('td').text.replace('\n','').strip())
                                                tech_status.append(table.td.find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.replace('\n','').strip())

                                            elif heading_set == head_set_1:
                                                make.append(None)
                                                model.append(None)
                                                participated_on.append(table.td.find_next('td').find_next('td').text)
                                                emd_status.append(table.td.find_next('td').find_next('td').find_next('td').text.replace('\n','').strip())
                                                tech_status.append(table.td.find_next('td').find_next('td').find_next('td').find_next('td').text.replace('\n',''))

                                            table = table.find_next('tr')


                                        table = inpage_soup.findAll("table",{"class":"table"})[1].tbody.tr 
                                        while(len(table)!=-0): 
                                            sn.append(table.td.text)
                                            seller_n.append(table.td.find_next('td').text.replace('\xa0\xa0\xa0Under PMA','').strip())
                                            offered_item.append(table.td.find_next('td').find_next('td').p.text.replace('Item Categories :','').strip())
                                            total_price.append(table.td.find_next('td').find_next('td').find_next('td').text)
                                            awarded.append(table.td.find_next('td').find_next('td').find_next('td').find_next('td').text)
                                            table = table.find_next('tr')
                                            in_page_bid_no_fin_eva.append(in_page_bid_no_now)
                                    else:
                                        print('Adding None In_Page')
                                        in_page_bid_no.append(None)
                                        in_page_bid_status.append(None)
                                        in_page_qty.append(None)
                                        in_page_bid_life_cycle.append(None)
                                        in_page_bid_validity.append(None)
                                        in_page_bid_start_Dt.append(None)
                                        in_page_bid_end_Dt.append(None)
                                        in_page_name.append(None)
                                        in_page_address.append(None)
                                        in_page_ministry.append(None)
                                        in_page_dept.append(None)
                                        in_page_orgnzn.append(None)
                                        in_page_office.append(None)
                            else: print('Added already')           
                        else: print('Status not Bid Award -',status)
                        print('Total No of Bids added = ',len(in_page_bid_no))      
                    else: print('RA Award')
                except ConnectionResetError:
                    print('CONNECTION RESET')
                    in_page_bid_no.append(None)
                    in_page_bid_status.append(None)
                    in_page_qty.append(None)
                    in_page_bid_life_cycle.append(None)
                    in_page_bid_validity.append(None)
                    in_page_bid_start_Dt.append(None)
                    in_page_bid_end_Dt.append(None)
                    in_page_name.append(None)
                    in_page_address.append(None)
                    in_page_ministry.append(None)
                    in_page_dept.append(None)
                    in_page_orgnzn.append(None)
                    in_page_office.append(None)
                    time.sleep(10)
                    pass
                except TimeoutError:
                    print('TIME OUT')
                    time.sleep(10)
                    #in_page_bid_no_now.append(None)
                    pass
        print('Im gonna sleep for sometime')
        #time.sleep(600)
        x = datetime.now()
        timenow = datetime(x.year,x.month,x.day,x.hour,x.minute,x.second)
        
except KeyboardInterrupt:
    print('Fine...Im gonna quit')

tech_eva = pd.DataFrame(list(zip(in_page_bid_no_tech_eva,sno,seller_name,make,model,participated_on,emd_status,tech_status)),columns = ['In Page Bid No','SNO','Seller Name','Make','Model','Participated On','EMD Status','Tecnical Status'])
fin_eva = pd.DataFrame(list(zip(in_page_bid_no_fin_eva,sn,seller_n,offered_item,total_price,awarded)),columns = ['In Page Bid No','SNO','Seller Name','Offered Item','Total Price','Awarded'])
bid_table = pd.DataFrame(list(zip(bid_no_true,bid_doc,bid_status,bid_item,qty_reqd,dept_addrs,start_date,end_date,bid_res,in_page_bid_no,in_page_bid_status,in_page_qty,in_page_bid_life_cycle,in_page_bid_validity,in_page_bid_start_Dt,in_page_bid_end_Dt,in_page_name,in_page_address,in_page_ministry,in_page_dept,in_page_orgnzn,in_page_office)), columns = ['Bid No','Bid Doc URL','Bid Status', 'Bid Item', 'Qty Reqd', 'Dept Address', 'Start Date', 'End Date', 'Bid Result', 'Inpage Bid No', 'Inpage_bid_status', 'Inpage Qty', 'Inpage Bid Life Cycle', 'Inpage Bid Validity' , 'Inpage Bid Start_Dt', 'Inpage Bid End_Dt' , 'Inpage Name', 'Inpage Address' , 'Inpage Ministry', 'Inpage dept', 'Inpage organization', 'Inpage Office'] )    


1
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
2
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
3
Status not Bid Award - Technical Evaluation
Total No of Bids added =  0
4
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
5
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
6
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
7
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
8
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
9
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
10
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
11
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
12
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
13
Status not Bid Award - Technical Evaluation
Total No of Bids added =  0
14
Status not Bid Award - Not Evaluated
Total No of Bids added =  0
15
Status not Bid Award - Not Evaluated
Tot

In [12]:
tech_eva = pd.DataFrame(list(zip(in_page_bid_no_tech_eva,sno,seller_name,make,model,participated_on,emd_status,tech_status)),columns = ['In Page Bid No','SNO','Seller Name','Make','Model','Participated On','EMD Status','Tecnical Status'])
fin_eva = pd.DataFrame(list(zip(in_page_bid_no_fin_eva,sn,seller_n,offered_item,total_price,awarded)),columns = ['In Page Bid No','SNO','Seller Name','Offered Item','Total Price','Awarded'])
bid_table = pd.DataFrame(list(zip(bid_no_true,bid_doc,bid_status,bid_item,qty_reqd,dept_addrs,start_date,end_date,bid_res,in_page_bid_no,in_page_bid_status,in_page_qty,in_page_bid_life_cycle,in_page_bid_validity,in_page_bid_start_Dt,in_page_bid_end_Dt,in_page_name,in_page_address,in_page_ministry,in_page_dept,in_page_orgnzn,in_page_office)), columns = ['Bid No','Bid Doc URL','Bid Status', 'Bid Item', 'Qty Reqd', 'Dept Address', 'Start Date', 'End Date', 'Bid Result', 'Inpage Bid No', 'Inpage_bid_status', 'Inpage Qty', 'Inpage Bid Life Cycle', 'Inpage Bid Validity' , 'Inpage Bid Start_Dt', 'Inpage Bid End_Dt' , 'Inpage Name', 'Inpage Address' , 'Inpage Ministry', 'Inpage dept', 'Inpage organization', 'Inpage Office'] )    

In [62]:
print('Bid table Shape = ',bid_table.shape)
print('Tech Eva Shape = ',tech_eva.shape)
print('Fin_eva Shape = ',fin_eva.shape)

Bid table Shape =  (33, 22)
Tech Eva Shape =  (277, 8)
Fin_eva Shape =  (33, 6)


In [53]:
bid_table

,Bid No,Bid Doc URL,Bid Status,Bid Item,Qty Reqd,Dept Address,Start Date,End Date,Bid Result,Inpage Bid No,...,Inpage Bid Life Cycle,Inpage Bid Validity,Inpage Bid Start_Dt,Inpage Bid End_Dt,Inpage Name,Inpage Address,Inpage Ministry,Inpage dept,Inpage organization,Inpage Office
0,GEM/2019/B/382165,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,stamp paper or bond paper,500,Ministry Of Defence Department Of Defence Prod...,10-10-2019 09:48:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382165,...,90 (Days),45 (Days),10-10-2019 09:48:25,21-10-2019 10:00:00,Prashantkumar Ramanlal Patel,"BHARAT ELECTRONICS LIMITED L-1, MIDC INDUSTRIA...",Ministry Of Defence,Department Of Defence Production,Bharat Electronics Limited (bel),Navi Mumbai
1,GEM/2019/B/382171,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,Air Conditioner,2,Ministry Of Defence Department Of Defence Kali...,10-10-2019 10:21:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382171,...,90 (Days),20 (Days),10-10-2019 10:21:16,21-10-2019 10:00:00,Krishan Kumar Pandey,"Krishan Kumar Pandey,MAHARASHTRA,kkpandey.194x...",Ministry Of Defence,Department Of Defence,Indian Army,Kalina Military Station East Mumbai 400029
2,GEM/2019/B/382141,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,Plain Copier Paper,200,Ministry Of Agriculture And Farmers Welfare De...,10-10-2019 09:02:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382141,...,90 (Days),30 (Days),10-10-2019 09:02:56,21-10-2019 10:00:00,Ram Prasad Pandey,"CFMTTI Tractor Nagar Budni,Ram Prasad Pandey,I...",Ministry Of Agriculture And Farmers Welfare,Department Of Agriculture Cooperation And Farm...,N/a,Central Farm Machinery Training &testing Insti...
3,GEM/2019/B/382144,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,binding punch machine,2,Ministry Of Defence Department Of Defence 105 ...,10-10-2019 09:14:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382144,...,90 (Days),30 (Days),10-10-2019 09:14:15,21-10-2019 10:00:00,Kunal Deshraj Singhal,"Kunal Deshraj Singhal,ASSAM,deepakjoshi.933p@g...",Ministry Of Defence,Department Of Defence,Indian Army,105 Engr Regt
4,GEM/2019/B/382145,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,purses and handbags and bags rucksack 70-85 l...,10,Ministry Of Defence Department Of Defence 151 ...,10-10-2019 09:14:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382145,...,90 (Days),30 (Days),10-10-2019 09:14:48,21-10-2019 10:00:00,Jamlal Guite,"Jamlal Guite,turmeric.pears@nic.in,781029,Indi...",Ministry Of Defence,Department Of Defence,Department Of Defence,151 Base Hospital
5,GEM/2019/B/382156,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,LED Luminaire (for Flood Light),5,Ministry Of Defence Department Of Defence Brig...,10-10-2019 09:28:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382156,...,90 (Days),30 (Days),10-10-2019 09:28:34,21-10-2019 10:00:00,Neetin Sehgal,"ARMY WAR COLLEGE MHOW CANTT,Neetin Sehgal,Indi...",Ministry Of Defence,Department Of Defence,Indian Army,Brig Adm
6,GEM/2019/B/382159,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,interactive panels and accessories,1,Ministry Of Defence Department Of Defence Indi...,10-10-2019 09:45:am,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...,GEM/2019/B/382159,...,90 (Days),30 (Days),10-10-2019 09:45:52,21-10-2019 10:00:00,Rajes Pal,"CGAE Porbandar, Near civil airport, Porbandar,...",Ministry Of Defence,Department Of Defence,Indian Coast Guard,Indian Coast Guard Air Enclave Porbandar
7,GEM/2019/B/381121,https://bidplus.gem.gov.in/showbidDocument/142...,Bid Award,Desktop Computers,1,Ministry Of Defence Department Of Defence Rese...,09-10-2019 12:15:pm,21-10-2019 10:00:am,https://bidplus.gem.gov.in/bidding/bid/getBidR...

In [63]:
p_bid_table = pd.read_excel("G:/Basil/Bid Table.xlsx")
p_tech_eva = pd.read_excel("G:/Basil/Technical Evaluation.xlsx")
p_fin_eva = pd.read_excel("G:/Basil/Financial Evaluation.xlsx")

bid_table = pd.concat([bid_table, p_bid_table], axis = 0)
bid_table = bid_table.drop_duplicates(keep=False)
tech_eva = pd.concat([tech_eva, p_tech_eva], axis = 0)
tach_eva = tech_eva.drop_duplicates(keep=False)
fin_eva = pd.concat([fin_eva,p_fin_eva], axis = 0)
fin_eva = fin_eva.drop_duplicates(keep=False)

bid_table.to_excel("G:/Basil/Bid Table.xlsx",index = False)
tech_eva.to_excel("G:/Basil/Technical Evaluation.xlsx",index = False)
fin_eva.to_excel("G:/Basil/Financial Evaluation.xlsx",index=False)

In [40]:
bid_table.to_excel("G:/Basil/Bid Table.xlsx",index = False)
tech_eva.to_excel("G:/Basil/Technical Evaluation.xlsx",index = False)
fin_eva.to_excel("G:/Basil/Financial Evaluation.xlsx",index=False)